# Spam Detector Project

in this project we aim to classify documents into spam and "non-spam" documents. in our approach we will use SpaCy library, bag of words (BoW) and a deep learning structure to predict the class of each document

In [48]:
# !pip install spacy
# !python -m spacy download en

import pandas as pd

# Loading the spam data
# ham is the label for non-spam messages
spam = pd.read_csv('../Text_Classification/spam.csv')
spam.head(10)
# print(spam.columns)
print(spam['label'][0])

ham


In [27]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" architecture
# config={"exclusive_classes": True,"architecture": "bow",}
config={"model":{"@architectures" : "spacy.TextCatBOW.v1",
"exclusive_classes" : True,
"ngram_size" : 1,
"no_output_layer" :False},}
textcat = nlp.add_pipe("textcat",config=config)

# textcat = nlp.add_pipe("textcat")
print(textcat.cfg)
# Add the TextCategorizer to the empty model
# nlp.add_pipe("textcat")

{'labels': [], 'threshold': 0.5, 'positive_label': None}


In [70]:
#Extra testing code
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL
print(DEFAULT_SINGLE_TEXTCAT_MODEL)
# Construction via add_pipe with custom model
config = {"model": {"@architectures": "spacy.TextCatBOW.v1","exclusive_classes":True,}}
textcat = nlp.add_pipe("textcat", config=config)

{'@architectures': 'spacy.TextCatEnsemble.v2', 'linear_model': {'@architectures': 'spacy.TextCatBOW.v1', 'exclusive_classes': True, 'ngram_size': 1, 'no_output_layer': False}, 'tok2vec': {'@architectures': 'spacy.Tok2Vec.v2', 'embed': {'@architectures': 'spacy.MultiHashEmbed.v1', 'width': 64, 'rows': [2000, 2000, 1000, 1000, 1000, 1000], 'attrs': ['ORTH', 'LOWER', 'PREFIX', 'SUFFIX', 'SHAPE', 'ID'], 'include_static_vectors': False}, 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2', 'width': 64, 'window_size': 1, 'maxout_pieces': 3, 'depth': 2}}}


ValueError: [E007] 'textcat' already exists in pipeline. Existing names: ['textcat']

In [28]:
# Add labels to text classifier
textcat.add_label("ham")
textcat.add_label("spam")

1

In [54]:
train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} 
                for label in spam['label']]
print(train_labels)

{'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': False, 'spam': True}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': False, 'spam': True}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, {'cats': {'ham': True, 'spam': False}}, 

In [30]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

In [32]:
import random
from spacy.util import minibatch
# V3
from spacy.tokens import Doc
from spacy.training import Example
# --------


spacy.util.fix_random_seed(1)
nlp.begin_training()
optimizer = nlp.initialize()
losses={}

for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=200)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        # v2
        # texts, labels = zip(*batch)
        # nlp.update(texts, labels, sgd=optimizer)
        # V3-------------------------------------
        examples = [Example.from_dict(nlp(text), cats) for text, cats in train_data]
        nlp.update(examples, sgd=optimizer, losses=losses)
        print(losses)

# v3 -----------------------------
# Training data for text categorization
# doc = nlp("I'm pretty happy about that!")
# gold_dict = {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0}}
# example = Example.from_dict(doc, gold_dict)

{'textcat': 1.6104520383919407e-08}
{'textcat': 3.1304915992791393e-08}
{'textcat': 4.5657970737522646e-08}
{'textcat': 5.922187007456614e-08}
{'textcat': 7.205341834293222e-08}
{'textcat': 8.420693387023448e-08}
{'textcat': 9.573365655768384e-08}
{'textcat': 1.066813712924386e-07}
{'textcat': 1.1709421965377942e-07}
{'textcat': 1.270126563923668e-07}
{'textcat': 1.3647353735990464e-07}
{'textcat': 1.455103060266083e-07}
{'textcat': 1.5415324483569748e-07}
{'textcat': 1.624297674140962e-07}
{'textcat': 1.7036471877673875e-07}
{'textcat': 1.779806675372697e-07}
{'textcat': 1.852981732497483e-07}
{'textcat': 1.9233603332224902e-07}
{'textcat': 1.9911148863016592e-07}
{'textcat': 2.056404135863943e-07}
{'textcat': 2.1193746757575127e-07}
{'textcat': 2.1801622995809566e-07}
{'textcat': 2.2388930975836274e-07}
{'textcat': 2.2956843936938753e-07}
{'textcat': 2.3506455315569497e-07}
{'textcat': 2.4038784962598925e-07}
{'textcat': 2.45547850941108e-07}
{'textcat': 2.505534486552108e-07}
{'text

In [ ]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

In [33]:
texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA" ]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)

[[0.99217105 0.00782895]
 [0.2827115  0.7172885 ]]


In [36]:
# From the scores, find the label with the highest score/probability
# predicted_labels = scores.argmax(axis=1)
import numpy as np
predicted_labels = np.argmax(scores, axis=1)

print([textcat.labels[label] for label in predicted_labels])

['ham', 'spam']
